In [1]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='BTC-Bitstamp-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Timestamp": "dateid", "Close": "price","Volume_(BTC)":"volume"})


df

,dateid,Open,High,Low,price,volume,Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [2]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume','High','Low'])
df

,price,ask,bid,volume,High,Low
date,,,,,,
2011-12-31 07:52:00,4.39,4.4339,4.3461,0.455581,4.39,4.39
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,58714.31,58686.00
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,58693.43,58683.97
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,58723.84,58693.43


In [3]:
#df = df.dropna()

def addVals(timeframe):
    p=df['price']
    df['ma'+timeframe]=(p/p.rolling(timeframe).mean())-1
    mins = df['Low'].rolling(timeframe).min()
    df['maxmin'+timeframe]=(p - mins)/(df['High'].rolling(timeframe).max() - mins) 
    df['std'+timeframe] =  p.rolling(timeframe).std()/p

addVals('15min')
addVals('1h')
addVals('10h')
addVals('1D')
addVals('10D')
addVals('20D')

df = df.dropna()

    
del df['High']
del df['Low']

df

,price,ask,bid,volume,ma15min,maxmin15min,std15min,ma1h,maxmin1h,std1h,...,std10h,ma1D,maxmin1D,std1D,ma10D,maxmin10D,std10D,ma20D,maxmin20D,std20D
date,,,,,,,,,,,,,,,,,,,,,
2011-12-31 17:00:00,4.58,4.6258,4.5342,9.000000,0.001093,1.000000,0.001544,0.001093,1.000000,0.001544,...,0.023338,0.021751,1.000000,0.023338,0.021751,1.000000,0.023338,0.021751,1.000000,0.023338
2012-01-03 17:14:00,5.14,5.1914,5.0886,0.020000,-0.014382,0.000000,0.020635,-0.014382,0.000000,0.020635,...,0.015515,-0.012623,0.437500,0.022629,0.046082,0.806452,0.067833,0.046082,0.806452,0.067833
2012-01-03 17:26:00,5.29,5.3429,5.2371,11.000000,0.014382,1.000000,0.020050,0.009542,1.000000,0.016371,...,0.014218,0.014139,0.906250,0.021121,0.071139,0.967742,0.066116,0.071139,0.967742,0.066116
2012-01-03 17:27:00,5.29,5.3429,5.2371,4.010815,0.009542,1.000000,0.016371,0.007139,1.000000,0.014178,...,0.013471,0.012548,0.906250,0.020296,0.066398,0.967742,0.065997,0.066398,0.967742,0.065997
2012-01-04 15:53:00,5.32,5.3732,5.2668,0.186090,0.012369,1.000000,0.017279,0.012369,1.000000,0.017279,...,0.012218,0.027821,1.000000,0.026777,0.062841,1.000000,0.059057,0.062841,1.000000,0.059057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,0.000311,0.530095,0.000776,0.000310,0.596089,0.000813,...,0.002117,0.005809,0.704474,0.011357,0.056279,0.922174,0.033753,0.036386,0.728955,0.036899
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,0.000229,0.528927,0.000744,0.000313,0.595137,0.000810,...,0.002115,0.005793,0.704394,0.011353,0.056275,0.922153,0.033754,0.036380,0.728938,0.036900
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,0.000736,0.762742,0.000707,0.000956,0.785811,0.000814,...,0.002111,0.006432,0.720389,0.011342,0.056959,0.926366,0.033734,0.037050,0.732268,0.036877
